In [2]:
import torch 
import transformers
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import AutoModelForCausalLM, AutoTokenizer
from data import get_e2e
from datasets import load_dataset

In [4]:
prompt_length = 4
model_type = "gpt2"

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_type)
lm_datasets = get_e2e(model_type, prompt_length)

Found cached dataset e2e_nlg (/home/ubuntu/.cache/huggingface/datasets/e2e_nlg/default/0.0.0/bfeceb720929c2705bd227d1cfe5eaaab102a0bdac10dad618dac1e00c737430)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/e2e_nlg/default/0.0.0/bfeceb720929c2705bd227d1cfe5eaaab102a0bdac10dad618dac1e00c737430/cache-382195a302ae4f32_*_of_00004.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/e2e_nlg/default/0.0.0/bfeceb720929c2705bd227d1cfe5eaaab102a0bdac10dad618dac1e00c737430/cache-91bd0eca8b749c90_*_of_00004.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/e2e_nlg/default/0.0.0/bfeceb720929c2705bd227d1cfe5eaaab102a0bdac10dad618dac1e00c737430/cache-21cc4b320de71e7c_*_of_00004.arrow


Map (num_proc=4):   0%|          | 0/42061 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4672 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4693 [00:00<?, ? examples/s]

In [4]:
lm_datasets.keys()

dict_keys(['train', 'validation', 'test'])

In [5]:
input_ids = lm_datasets['test'][0]['input_ids']

In [6]:
labels = lm_datasets['test'][0]['labels']

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_type, use_fast=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [8]:
tokenizer.decode(input_ids)

'!!!!!!!!!name[Blue Spice], eatType[coffee shop], area[city centre]name[Blue Spice], eatType[coffee shop], area[city centre]name[Blue Spice], eatType[coffee shop], area[riverside]name[Blue Spice], eatType[coffee shop], area[riverside]name[Blue Spice], eatType[coffee shop], customer rating[5 out of 5], near[Crowne Plaza Hotel]name[Blue Spice], eatType[coffee shop], customer rating[5 out of 5], near[Crowne Plaza Hotel]name[Blue Spice], eatType[coffee shop], customer rating[5 out of 5], near[Crowne Plaza Hotel]name[Blue Spice], eatType[coffee shop], customer rating[average], near[Burger King]name[Blue Spice], eatType[coffee shop], customer rating[average], near[Burger King]name[Blue Spice], eatType[coffee shop], customer rating[average], near[Burger King]name[Blue Spice], eatType[coffee shop], customer rating[average], near[Burger King]name[Blue Spice], eatType[coffee shop], customer rating[average], near[Burger King]name[Blue Spice], eatType[coffee shop], customer rating[average], near[B

In [9]:
tokenizer.decode([0 if x < 0 else x for x in labels])

'!!!!!!!!!name[Blue Spice], eatType[coffee shop], area[city centre]name[Blue Spice], eatType[coffee shop], area[city centre]name[Blue Spice], eatType[coffee shop], area[riverside]name[Blue Spice], eatType[coffee shop], area[riverside]name[Blue Spice], eatType[coffee shop], customer rating[5 out of 5], near[Crowne Plaza Hotel]name[Blue Spice], eatType[coffee shop], customer rating[5 out of 5], near[Crowne Plaza Hotel]name[Blue Spice], eatType[coffee shop], customer rating[5 out of 5], near[Crowne Plaza Hotel]name[Blue Spice], eatType[coffee shop], customer rating[average], near[Burger King]name[Blue Spice], eatType[coffee shop], customer rating[average], near[Burger King]name[Blue Spice], eatType[coffee shop], customer rating[average], near[Burger King]name[Blue Spice], eatType[coffee shop], customer rating[average], near[Burger King]name[Blue Spice], eatType[coffee shop], customer rating[average], near[Burger King]name[Blue Spice], eatType[coffee shop], customer rating[average], near[B

In [10]:
inp = tokenizer.encode("My name is luke. My name is")

In [11]:
out = model(torch.tensor(inp))

In [12]:
tokenizer.decode(out['logits'].argmax(axis=1))

'\n is Johnu. I name is l'

In [9]:
datasets = load_dataset("e2e_nlg")

Found cached dataset e2e_nlg (/home/ubuntu/.cache/huggingface/datasets/e2e_nlg/default/0.0.0/bfeceb720929c2705bd227d1cfe5eaaab102a0bdac10dad618dac1e00c737430)


  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
datasets.keys()

dict_keys(['train', 'validation', 'test'])

In [15]:
len(datasets['train'])

42061

In [16]:
tokenizer.bos_token

'<|endoftext|>'

In [12]:
def tokenize_e2e(examples):
    """Helper function for tokenizing e2e dataset"""

    prompt = [
        "Summarize: " + x + tokenizer.bos_token + y + tokenizer.eos_token 
        for x, y in zip(examples["meaning_representation"], examples["human_reference"])
    ]
    d = {
        "input_ids": tokenizer(prompt, padding=True)["input_ids"],
        "attention_mask": tokenizer(prompt, padding=True)["attention_mask"],
        "labels": tokenizer(prompt, padding=False)["input_ids"],
    }

    return d

In [13]:
tokenized_datasets = datasets.map(
    tokenize_e2e,
    num_proc=4,
    batched=True,
    remove_columns=["meaning_representation", "human_reference"],
)

Map (num_proc=4):   0%|          | 0/42061 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4672 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4693 [00:00<?, ? examples/s]

In [14]:
tokenizer.decode(tokenized_datasets["train"][0]["input_ids"])

'Summarize: name[The Vaults], eatType[pub], priceRange[more than £30], customer rating[5 out of 5], near[Café Adriatic]<|endoftext|>The Vaults pub near Café Adriatic has a 5 star rating.  Prices start at £30.<|endoftext|>[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [15]:
def add_prompt_dummy_tokens(examples):
    """Helper function to group short examples together for faster training. Note
    This is not necessary, and is only used to accelerate training"""

    dummy_input_prefix = [0] * prompt_len
    dummy_label_prefix = [-100] * prompt_len
    dummy_attention_mask_prefix = [1] * prompt_len

    # Split by chunks of max_len.
    input_ids = examples["input_ids"]
    attention_masks = examples["attention_mask"]
    labels = examples["labels"]
    result = {}

    result["input_ids"] = [
        dummy_input_prefix + x for x in input_ids
    ]
    result["labels"] = [
        dummy_label_prefix + x for x in labels
    ]
    result["attention_mask"] = [
        dummy_attention_mask_prefix + x for x in attention_masks
    ]

    return result

In [17]:
prompt_len = 4
custom_datasets = tokenized_datasets.map(
    add_prompt_dummy_tokens,
    batched=True,
    num_proc=4,
)

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/e2e_nlg/default/0.0.0/bfeceb720929c2705bd227d1cfe5eaaab102a0bdac10dad618dac1e00c737430/cache-0b14d2f1e8a69a8b_*_of_00004.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/e2e_nlg/default/0.0.0/bfeceb720929c2705bd227d1cfe5eaaab102a0bdac10dad618dac1e00c737430/cache-d5d36f6a7e3d1f21_*_of_00004.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/e2e_nlg/default/0.0.0/bfeceb720929c2705bd227d1cfe5eaaab102a0bdac10dad618dac1e00c737430/cache-b1cb4c6a65121800_*_of_00004.arrow


In [127]:
max_ = 0
for key in lm_datasets.keys():
    dataset = lm_datasets[key]
    for x in dataset:
        if len(x["input_ids"]) > max_:
            max_ = len(x["input_ids"])

In [128]:
max_

150

In [120]:
for x in lm_datasets["test"]:
    assert(len(x["input_ids"]) == len(x["attention_mask"]))
    assert(len(x["input_ids"]) == len(x["labels"]))

In [18]:
def evaluate_gpt2_with_prefix_2(
    model_checkpoint="test-clm/checkpoint-243", 
    eval_step=100, 
    prompt_len=1, 
    model_type="gpt2",
    lm_datasets=None
):

    # Load model and freeze all parameters
    model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
    for param in model.parameters():
        param.requires_grad = False

    # load e2e dataset
    if lm_datasets is None:
        lm_datasets = get_e2e(model_type, prompt_len)

    # Setup Trainer just to use predict interface
    trainer = Trainer(
        model,
        args=TrainingArguments("test-clm", per_device_eval_batch_size=1),
    )

    # Run evaluation in batches of training set to avoid GPU OOM
    N = len(lm_datasets["test"])
    predictions = []
    for k in range(0, N, eval_step):
        window = range(k, min(k+eval_step, N))
        preds = trainer.predict(lm_datasets["test"].select(window), metric_key_prefix="test_bleu")
        predictions.append(preds)
 
    #Calculate average BLEU metric across sets
    average_bleu = 0
    bleu_vals = []
    for p in predictions:
        bleu = p[2]['test_bleu_loss']
        average_bleu += bleu
        bleu_vals.append(bleu)

    print(bleu_vals)
    print("Average Bleu: ", average_bleu/len(predictions))

In [19]:
evaluate_gpt2_with_prefix_2("gpt2", 50, 0, "gpt2", custom_datasets)

***** Running Prediction *****
  Num examples = 50
  Batch size = 1
/opt/conda/conda-bld/pytorch_1670525539683/work/aten/src/ATen/native/cuda/Indexing.cu:1141: indexSelectLargeIndex: block: [480,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1670525539683/work/aten/src/ATen/native/cuda/Indexing.cu:1141: indexSelectLargeIndex: block: [480,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1670525539683/work/aten/src/ATen/native/cuda/Indexing.cu:1141: indexSelectLargeIndex: block: [480,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1670525539683/work/aten/src/ATen/native/cuda/Indexing.cu:1141: indexSelectLargeIndex: block: [480,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1670525539683/work/aten/src/ATen/native/cuda/Indexing.cu:1141: indexSelectLargeIndex: block: [480,0,0], thread: [4,

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [20]:
from evaluate import evaluate_gpt2_with_prefix

In [21]:
evaluate_gpt2_with_prefix("gpt2", 50, 0, "gpt2")

loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transforme

  0%|          | 0/3 [00:00<?, ?it/s]

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.